## Sampling Experiments

First goal is to aggregate the Wisconsin voting data into different levels of resolution, ideally using these specifications: https://people.csail.mit.edu/ddeford/BLOCK/55.html

In [3]:
#custom build tools
from graph_tools import graph_from_url
#general python tools
import csv
import os
from functools import partial
import json
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
from networkx import is_connected, connected_components
#gerry chain tools 
from gerrychain import (
    Election,
    Graph,
    MarkovChain,
    Partition,
    accept,
    constraints,
    updaters,
    GeographicPartition,
)

In [6]:
df = geopandas.read_file("./data/Wisconsin/WI_ltsb_corrected_final.shp")
df.set_index("GEOID10", inplace=True)
graph = Graph.from_geodataframe(df)
graph.add_data(df)

/usr/local/lib/python3.7/site-packages/gerrychain/graph/graph.py:228: UserWarning: Found islands (degree-0 nodes). Indices of islands: {'55029836000001', '55003425620001'}
  "Found islands (degree-0 nodes). Indices of islands: {}".format(islands)
